In [ ]:
import pandas as pd
import numpy as np
import warnings
import itertools
import plotly.plotly as ply
from math import sqrt
from matplotlib import pyplot as plt
from sklearn.metrics import mean_squared_error

warnings.filterwarnings("ignore")

In [ ]:
df = pd.read_csv("House1_1min_temperature.csv",index_col = 0)
df.head()
df.index = pd.to_datetime(df.index)
df = df.resample("1H").sum()
df.head()


In [ ]:
df['Timing'] = df.index.hour

In [ ]:
#df['Total'] = df.iloc[:,0:7].sum(axis=1)

In [ ]:
df.head()

In [ ]:
df_1 = pd.DataFrame(df["Freezer"])
df_2 = pd.DataFrame(df["Tmax"]//60)
df_3 = pd.DataFrame(df["Tmin"]//60)
df_4 = pd.DataFrame(df["Timing"])
df_5 = pd.DataFrame(df["Fridge"])
df_6 = pd.DataFrame(df["Temp"]//60)
freezer_temp = pd.concat([df_1,df_2,df_3,df_4,df_5,df_6], axis = 1)


In [ ]:
freezer_temp.head()

In [ ]:
from math import sqrt
from numpy import concatenate
from matplotlib import pyplot
from pandas import read_csv
from pandas import DataFrame
from pandas import concat
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM

In [ ]:
# convert series to supervised learning
def series_to_supervised(data, n_in=1, n_out=1, dropnan=True):
    n_vars = 1 if type(data) is list else data.shape[1]
    df = DataFrame(data)
    cols, names = list(), list()
    # input sequence (t-n, ... t-1)
    for i in range(n_in, 0, -1):
        cols.append(df.shift(i))
        names += [('var%d(t-%d)' % (j+1, i)) for j in range(n_vars)]
    # forecast sequence (t, t+1, ... t+n)
    for i in range(0, n_out):
        cols.append(df.shift(-i))
        if i == 0:
            names += [('var%d(t)' % (j+1)) for j in range(n_vars)]
        else:
            names += [('var%d(t+%d)' % (j+1, i)) for j in range(n_vars)]
    # put it all together
    agg = concat(cols, axis=1)
    agg.columns = names
    # drop rows with NaN values
    if dropnan:
        agg.dropna(inplace=True)
    return agg

In [ ]:
# ensure all data is float
values = freezer_temp.astype('float32')
# normalize features
scaler = MinMaxScaler(feature_range=(0, 1))
scaled = scaler.fit_transform(values)
#frame as supervised learning
reframed = series_to_supervised(scaled, 1, 1)
reframed.head()

In [ ]:
reframed2 = reframed.copy()

In [ ]:
reframed.drop(['var2(t)','var3(t)','var6(t)','var4(t)','var5(t)'],axis=1, inplace=True)
reframed.head(10)

In [ ]:
# split into train and test sets
values = reframed.values
size = int(len(freezer_temp["Freezer"]) * 0.75)
train = values[:size, :]
test = values[size:, :]
# split into input and outputs
train_X, train_y = train[: , :-1], train[:, -1]
test_X, test_y = test[:, :-1], test[:, -1]

In [ ]:
# reshape input to be 3D [samples, timesteps, features]
train_X = train_X.reshape((train_X.shape[0], 1, train_X.shape[1]))
test_X = test_X.reshape((test_X.shape[0], 1, test_X.shape[1]))
print(train_X.shape, train_y.shape, test_X.shape, test_y.shape)

In [ ]:
# design network
model = Sequential()
model.add(LSTM(6, input_shape=(train_X.shape[1], train_X.shape[2])))
model.add(Dense(1))
model.compile(loss='mae', optimizer='adam')

In [ ]:
# fit network
history = model.fit(train_X, train_y, epochs=10, batch_size=1, validation_data=(test_X, test_y), verbose=2, shuffle=False)

In [ ]:
# plot history
pyplot.plot(history.history['loss'], label='train')
pyplot.plot(history.history['val_loss'], label='test')
pyplot.legend()
pyplot.show()

In [ ]:
# make a prediction
yhat = model.predict(test_X)
test_X2 = test_X.reshape((test_X.shape[0], test_X.shape[2]))
inv_yhat = concatenate((yhat, test_X2[:, 1:]), axis=1)
inv_yhat = scaler.inverse_transform(inv_yhat)
inv_yhat = inv_yhat[:,0]
#test_X = test_X.reshape((test_X.shape[0], test_X.shape[2]))


In [ ]:
inv_yhat

In [ ]:
from plotly import __version__
%matplotlib inline
import cufflinks as cf
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot 
init_notebook_mode(connected=True)

init_notebook_mode(connected=True)
cf.go_offline()

#plot the actual consumption and the prediction 
future_forecast = pd.DataFrame(inv_yhat,index = freezer_temp.index[size + 1:],columns=['Prediction'])
pd.concat([freezer_temp["Freezer"],future_forecast],axis=1).iplot()

In [ ]:
pred = list(inv_yhat)
actual = list(freezer_temp["Freezer"])

In [ ]:
error = []
for i in range(len(pred)):
    error.append(actual[i] - pred[i])

In [ ]:
plt.hist(error)

In [ ]:
error = pd.DataFrame(error,index = freezer_temp.index[size + 1:],columns=['error'])
error.iplot()


In [ ]:
# invert scaling for forecast
inv_yhat = concatenate((yhat, test_X[:, 1:]), axis=1)
inv_yhat.shape

In [ ]:
inv_yhat = scaler.inverse_transform(inv_yhat)
inv_yhat = inv_yhat[:,0]

In [ ]:
def mean_absolute_percentage_error(y_true, y_pred): 
    l = []
    for i in range(len(y_true)):
        a = abs(y_true[i] - y_pred[i])
        b = a/y_true[i]
        l.append(b)
    print(sum(l))
    print(len(l))
    return (sum(l)/len(l))*100

In [ ]:
mean_absolute_percentage_error(freezer_temp["Freezer"][size + 1:], inv_yhat)